# Converting Raw and Result Data to Matlab

6/1/19: Need to convert raw and resulting data to matlab.

* raw.fif + .json -> .mat file
* raw.npz + .json -> .mat file

Here, I want to allow for cross-platform data compatability to allow for easy computing on either platforms.

The goal will be to test a variety of possible algorithms developed in matlab such as:

1. TVART
2. MR.SID

In [1]:
import os
import sys
import numpy as np
import scipy.io

%matplotlib inline
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

sys.path.append("../../")
from eztrack.edp.loaders.patient.subjectresultsloader import SubjectResultsLoader
from eztrack.edp.loaders.dataset.result.resultloader import ResultLoader
from eztrack.edp.loaders.dataset.timeseries.ieegrecording import iEEGRecording
from eztrack.edp.loaders.dataset.timeseries.scalprecording import ScalpRecording


# Import magic commands for jupyter notebook 
%load_ext autoreload
%autoreload 2

# Set Data Directories

Note here that resultsdir is somehow organized, but it doesn't have to be as long as the datafiles are within resultsdir/

In [2]:
center = 'nih'
reference = 'common_avg'
modality = 'ieeg'
patid = 'pt1'
datasetid = "sz_2"

DATADIR = '/Users/adam2392/Downloads/output_new/'

rawdatadir = f"/Users/adam2392/Downloads/tngpipeline/{center}/{patid}/seeg/fif/"
resultsdir = os.path.join(DATADIR,
                          'fragility',
                          reference,
                          modality, 
                          center)

jsonfilepath = f"{patid}_{datasetid}.json"
resultjsonfile = f"{patid}_{datasetid}_frag.json"

### Dataset

In [3]:
datasetloader = ResultLoader(jsonfilepath=os.path.join(resultsdir, resultjsonfile),
                            preload=True,
                            datatype='frag',
                            results_dir=resultsdir)

print("Finished loading in data for this patient")

Loading results data from:  /Users/adam2392/Downloads/output_new/fragility/common_avg/ieeg/nih/pt1_sz_2_fragmodel.npz
Finished loading in data for this patient


In [4]:
print("These are all the datasets: \n", datasetloader.result)

These are all the datasets: 
 pt1 sz_2 Fragility Model (86, 2151)


In [3]:
print(rawdatadir)
if modality == 'seeg' or modality == 'ieeg':
    loader = iEEGRecording(root_dir=rawdatadir,
                      reference=reference)

elif modality == 'scalp':
    loader = ScalpRecording(root_dir=rawdatadir,
                           reference=reference)

# select a json filepath to run
jsonfilepaths = loader.jsonfilepaths

print(jsonfilepaths)

/Users/adam2392/Downloads/tngpipeline/nih/pt1/seeg/fif/
reference is  common_avg
['pt1_ii_1.json', 'pt1_ii_2.json', 'pt1_sz_2.json', 'pt1_sz_3.json', 'pt1_sz_4.json', 'pt1_sz_6.json']


In [4]:
ieegts = loader.loadpipeline(jsonfilepath)

hard-code setting line freq to 60.
Set line frequency as: 60
filtering at:  [ 60. 120. 180. 240. 300. 360. 420. 480.]  and  [0.5, 499.0]
dict_keys(['ablated_contacts', 'age_surgery', 'bad_channels', 'channeltypes', 'clinical_center', 'clinical_difficulty', 'clinical_match', 'clinical_semiology', 'dataset_id', 'date_of_recording', 'edffilename', 'engel_score', 'equipment', 'events', 'ez_elecs', 'ez_hypo_brainregion', 'ez_hypo_contacts', 'filename', 'gender', 'hand_dominant', 'length_of_recording', 'non_eeg_channels', 'note', 'number_chans', 'onset', 'onset_age', 'outcome', 'patient_id', 'resect_elecs', 'resected_contacts', 'seizure_semiology', 'termination', 'type', 'samplerate', 'chanlabels', 'modality', 'lowpass_freq', 'highpass_freq', 'linefreq', 'onsetsec', 'offsetsec', 'onsetind', 'offsetind', 'reference'])
Still has white matter contacts:  (86, 269080)
Removed wm contacts:  (86,) (86, 269080)
These contacts are identified wm contacts:  []
86
Removed white matter contacts:  (86, 26

In [5]:
print(ieegts.reference)

common_avg


# 2. Using the Data

Note that the datasets are organized in python classes, so each result loaded is a list of these FragilityModelResult classes. See /edp/base/objects/dataset/result_object.py

This class helps user to extract out metadata and reformat data correctly.

Note for scalp data, cezinds returns an empty list, while for ieeg, cezlobeinds will return an empty list.

In [6]:
# assert data is save
rawdata = ieegts.get_data()
metadata = ieegts.get_metadata()

In [7]:
print(rawdata.shape)
print(ieegts.chanlabels)

(86, 269080)
['g1' 'g2' 'g3' 'g4' 'g5' 'g6' 'g7' 'g8' 'g9' 'g10' 'g13' 'g14' 'g15'
 'g16' 'g17' 'g18' 'g19' 'g20' 'g21' 'g22' 'g23' 'g11' 'g12' 'g24' 'g25'
 'g26' 'g27' 'g28' 'g29' 'g30' 'g31' 'g32' 'att1' 'att2' 'att3' 'att4'
 'att5' 'att6' 'att7' 'att8' 'plt1' 'plt2' 'plt3' 'plt4' 'plt5' 'plt6'
 'ast1' 'ast2' 'ast3' 'ast4' 'pst1' 'pst2' 'pst3' 'pst4' 'ad1' 'ad2' 'ad3'
 'ad4' 'pd1' 'pd2' 'pd3' 'pd4' 'sf1' 'sf2' 'sf3' 'sf4' 'sf5' 'sf6' 'if1'
 'if2' 'if3' 'if4' 'if5' 'if6' 'ilt1' 'ilt2' 'ilt3' 'ilt4' 'mlt1' 'mlt2'
 'mlt3' 'mlt4' 'slt1' 'slt2' 'slt3' 'slt4']


In [9]:
exdataset = datasetloader.result
print(exdataset)

# compute how to separate channels into their lobes
# exdataset.compute_montage_groups()

pt1 sz_2 Fragility Model (86, 2151)


In [21]:
ltvmat = exdataset.ltvmodel.get_data()
pertmat = exdataset.pertmodel.get_data()

print(ltvmat.shape, pertmat.shape, len(metadata['chanlabels']))

(86, 86, 2151) (86, 2151) 86


In [10]:
# get the fragility matrix
fragmat = exdataset.get_data()

# get the corresponding channels labels
chanlabels = exdataset.chanlabels

# get ieeg ez
cezcontacts = exdataset.cezcontacts

# get lobe
# cezlobe = exdataset.cezlobe
# get indices of ieeg cez
ieegcezinds = exdataset.cezinds
# cezinds = exdataset.cezlobeinds

print(fragmat.shape, len(chanlabels))

# print(cezcontacts, cezlobe, ieegcezinds, cezinds)

(86, 2151) 86


# 3. Convert Raw/Result Data To Mat Files

In [17]:
rawdatamat_path = os.path.join(f"/Users/adam2392/Downloads/tngpipeline/{center}/{patid}/", 
                               f'{patid}_{datasetid}.mat')

resultdatamat_path = os.path.join(f"/Users/adam2392/Downloads/tngpipeline/{center}/{patid}/", 
                               f'{patid}_{datasetid}_fragmodel.mat')

print(rawdatamat_path)
print(resultdatamat_path)

/Users/adam2392/Downloads/tngpipeline/nih/pt1/pt1_sz_2.mat
/Users/adam2392/Downloads/tngpipeline/nih/pt1/pt1_sz_2_fragmodel.mat


In [8]:
# save rawdata as .mat file
rawdatamat_path = "/Users/adam2392/Downloads/pt1_sz_2_rawdata_cavg.mat"
print('Saving rawdata to %s' % (rawdatamat_path))
scipy.io.savemat(rawdatamat_path, {'rawdata': rawdata,
                                   'chlabels': metadata['chanlabels'],
                                  }
                )

Saving rawdata to /Users/adam2392/Downloads/pt1_sz_2_rawdata_cavg.mat


In [22]:
# save results as .mat file
print('Saving rawdata to %s' % (rawdatamat_path))
scipy.io.savemat(rawdatamat_path, {'ltvmat': ltvmat,
                                   'pertmat': pertmat,
                                   'chlabels': metadata['chanlabels'],
                                  }
                )

Saving rawdata to /Users/adam2392/Downloads/tngpipeline/nih/pt1/pt1_sz_2.mat
